In [1]:
!pip install --user mglearn
!pip install -U openml
!pip install imbalanced-learn

  Obtaining dependency information for openml from https://files.pythonhosted.org/packages/4a/f3/462c16d9e80177d22a036abd3df0f114bf311e566ff906adbb3d82790c20/openml-0.15.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 293.2 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openml
    Found existing installation: openml 0.14.1
    Uninstalling openml-0.14.1:
      Successfully uninstalled openml-0.14.1


In [3]:
# Import necessary packages, modules, and functions
import numpy as np
import os
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import mglearn
from sklearn.tree import DecisionTreeClassifier
import pandas as pd 
from sklearn.linear_model import LogisticRegression
import seaborn as sns 
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings('ignore')
import openml
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline

ImportError: cannot import name 'ProgressType' from 'minio.helpers' (/Users/christianmotta/anaconda3/lib/python3.11/site-packages/minio/helpers.py)

In [16]:
## Loading data
#5645 sample: 12 features

diamonds = np.load('data.npz')
X = diamonds['features']
y = diamonds['labels']

#Change np to DataFrame
df= pd.DataFrame(X)

FileNotFoundError: [Errno 2] No such file or directory: 'data.npz'

In [4]:
#Preprocessing 

#Link used for the code for median: https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b

#Check for missing values

print("Number of missing values for every feature: \n{}".format(df.isnull().sum()))

print("Total of missing value in dataset: {}".format(df.isnull().sum().sum()))

#Replace missing values with median in column
for i in range(12):
    median= df[i].median()
    df[i].fillna(median, inplace= True)
    
print("New number of missing values for every feature: \n{}".format(df.isnull().sum()))

print("new total of missing value in dataset: {}".format(df.isnull().sum().sum()))
    
#Check for imbalanced data:
print("Number of samples and features:", X.shape)
print("Samples per class:", np.unique(y, return_counts=True))

Number of missing values for every feature: 
0     288
1     301
2     276
3     274
4     288
5     285
6     266
7     286
8     287
9     272
10    287
11    277
dtype: int64
Total of missing value in dataset: 3387
New number of missing values for every feature: 
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64
new total of missing value in dataset: 0
Number of samples and features: (5645, 12)
Samples per class: (array([0, 1, 2, 3], dtype=int32), array([1272,  402, 1447, 2524]))


In [5]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

In [6]:
#Decision Tree model
dt= DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(dt.score(X_train, y_train)))

#display the tree depth 
print(dt.get_depth())

Accuracy on training set: 0.999
22


In [7]:
#Hyperparameter tuning decision tree

#For tuning the hyperparameter and cross validation I used this link:https://plainenglish.io/blog/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda
#For SMOTE to implement it with pipeline and pipeline: https://towardsdatascience.com/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7

#Pipeline for processing data decision tree
sm = SMOTE()
pipeline_dt= imbpipeline([
    ('smote', sm),
    ('classifier', DecisionTreeClassifier(random_state=0))])                                       

#values
param_dict_dt = {
    "classifier__max_depth":range(2,13,3),
    "classifier__min_samples_split": range(2,12,2),
    "classifier__min_samples_leaf": range(2,7,2)
}

#Finding best hyperparameter/ cross validation
    
grid_dt= GridSearchCV(estimator=pipeline_dt, param_grid=param_dict_dt, cv=10)                  
grid_dt.fit(X_train, y_train)
                   
#Best parameters/ Evaluation 
print("This are the best parameters=",grid_dt.best_params_)
print("Cross-validation score=",grid_dt.best_score_)

This are the best parameters= {'classifier__max_depth': 8, 'classifier__min_samples_leaf': 6, 'classifier__min_samples_split': 2}
Cross-validation score= 0.6920729259395019


In [8]:
#Hyperparameter tuning Logistic Regression

#For SMOTE/ standardscaler to implement it with pipeline and cross validation: https://towardsdatascience.com/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7

pipeline_lr= imbpipeline([
    ('scaler', StandardScaler()),
    ('smote', sm),
    ('classifier', LogisticRegression(random_state=0))])          

#values
param_dict_lr = {
    "classifier__C":[0.01, 0.10, 1,10,100],
    "classifier__solver":["newton-cg", "lbfgs", "sag", "saga"]
}

#Finding best hyperparameter/ cross validation
    
grid_lr= GridSearchCV(estimator=pipeline_lr, param_grid=param_dict_lr, cv=10)                  
grid_lr.fit(X_train, y_train)
                   
#Best parameters/ Evaluation 
print("This are the best parameters=",grid_lr.best_params_)
print("Cross-validation score=",grid_lr.best_score_)

This are the best parameters= {'classifier__C': 0.1, 'classifier__solver': 'lbfgs'}
Cross-validation score= 0.7068361522752264


In [9]:
# Model comparison
import pandas

#Train time Decision Tree
traintime_dt= pandas.DataFrame(grid_dt.cv_results_)

print( "The average train time of the decision tree:",sum(traintime_dt['mean_fit_time'])/ len(traintime_dt['mean_fit_time']))

#Train time logistic regression
traintime_lr= pandas.DataFrame(grid_lr.cv_results_)

print( "The average train time of the logistic regression tree:",sum(traintime_lr['mean_fit_time'])/ len(traintime_lr['mean_fit_time']))

The average train time of the decision tree: 0.07652028481165568
The average train time of the logistic regression tree: 0.13210734963417053


In [10]:
#Model evaluation comparison

#Model performances

#Decision Tree
print("Model performance of the decision tree: {}".format(grid_dt.best_score_))

#Logistic Regression
print("Model performance of the logistic regression : {}".format(grid_lr.best_score_))

# Best Model
if grid_lr.best_score_> grid_dt.best_score_:
    print("The logistic regression has the best performance.")
else: 
    ("The decision tree has the best performance.")

Model performance of the decision tree: 0.6920729259395019
Model performance of the logistic regression : 0.7068361522752264
The logistic regression has the best performance.


In [11]:
#Final test
print("Final test set score for logistic regression: {:.2f}".format(grid_lr.score(X_test, y_test)))

Final test set score for logistic regression: 0.68


In [12]:
#Final results
pred_lr = grid_lr.predict(X_test)

confusion = confusion_matrix(y_test, pred_lr)
print("Confusion matrix:\n{}".format(confusion))
print()

print(classification_report(y_test, pred_lr, target_names=["Lab-grown", "Zirconia","Moissanite","Natural" ]))

Confusion matrix:
[[205   3  16  46]
 [  2  61   0   0]
 [ 34   0 209  49]
 [101  30  80 293]]

              precision    recall  f1-score   support

   Lab-grown       0.60      0.76      0.67       270
    Zirconia       0.65      0.97      0.78        63
  Moissanite       0.69      0.72      0.70       292
     Natural       0.76      0.58      0.66       504

    accuracy                           0.68      1129
   macro avg       0.67      0.76      0.70      1129
weighted avg       0.69      0.68      0.68      1129

